In [20]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import mean_squared_error

In [72]:
df = pd.read_csv("../data/03/housing/housing_train.csv")
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.13960,0.0,8.56,0.0,NaN,6.167,90.0,2.4210,5,384.0,NaN,392.69,NaN,86.066034
1,0.03510,95.0,2.68,0.0,NaN,7.853,33.2,5.1180,4,224.0,NaN,392.78,NaN,207.700057
2,15.87440,0.0,18.10,0.0,NaN,6.545,99.1,1.5192,24,666.0,NaN,396.90,21.08,46.689349
3,0.18337,0.0,27.74,0.0,NaN,5.414,98.3,1.7554,4,711.0,NaN,344.05,23.97,29.980669
4,0.12816,12.5,6.07,0.0,NaN,5.885,33.0,6.4980,4,345.0,NaN,396.90,NaN,89.461275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0.17120,0.0,8.56,0.0,NaN,5.836,91.9,2.2110,5,384.0,20.9,395.67,18.66,83.647456
426,0.29916,20.0,6.96,0.0,NaN,5.856,42.1,4.4290,3,223.0,18.6,388.65,NaN,90.409128
427,0.01501,80.0,2.01,0.0,NaN,6.635,29.7,8.3440,4,280.0,17.0,390.94,NaN,104.921869
428,11.16040,0.0,18.10,0.0,0.74,6.629,94.6,2.1247,24,666.0,20.2,109.85,23.27,57.427501


In [73]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [74]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     430 non-null    float64
 1   ZN       430 non-null    float64
 2   INDUS    430 non-null    float64
 3   CHAS     408 non-null    float64
 4   NOX      52 non-null     float64
 5   RM       430 non-null    float64
 6   AGE      430 non-null    float64
 7   DIS      430 non-null    float64
 8   RAD      430 non-null    int64  
 9   TAX      420 non-null    float64
 10  PTRATIO  420 non-null    float64
 11  B        430 non-null    float64
 12  LSTAT    88 non-null     float64
 13  MEDV     430 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 47.2 KB


In [75]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,430.000000,430.000000,430.000000,408.000000,52.000000,430.000000,430.000000,430.000000,430.000000,420.000000,420.000000,430.000000,88.000000,430.000000
mean,3.694840,11.370930,11.157209,0.073529,0.772442,6.298342,68.916744,3.772554,9.504651,410.959524,18.371429,354.476256,23.902386,97.222903
std,8.789114,23.197997,6.934849,0.261324,0.065919,0.717532,28.019891,2.131754,8.680752,168.251675,2.214768,93.897035,4.756535,40.155983
min,0.009060,0.000000,0.740000,0.000000,0.713000,3.561000,2.900000,1.129600,1.000000,188.000000,12.600000,0.320000,17.930000,21.445934
25%,0.081900,0.000000,5.130000,0.000000,0.713000,5.888250,45.450000,2.063900,4.000000,283.250000,16.900000,374.597500,19.910000,72.899822
50%,0.274475,0.000000,9.125000,0.000000,0.740000,6.210000,78.200000,3.107300,5.000000,332.000000,18.800000,391.305000,23.255000,91.648354
75%,3.560263,12.500000,18.100000,0.000000,0.871000,6.630750,93.975000,5.269125,24.000000,666.000000,20.200000,395.935000,26.930000,108.098012
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,214.412960


In [76]:
df["NOX"] = df.NOX.isnull().astype("int")
df["LSTAT"] = df.LSTAT.isnull().astype("int")

In [77]:
cormat = df.corr()
round(cormat,2)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
CRIM,1.00,-0.20,0.40,-0.05,-0.11,-0.21,0.34,-0.37,0.63,0.58,0.29,-0.38,-0.41,-0.38
ZN,-0.20,1.00,-0.53,-0.04,0.18,0.28,-0.58,0.67,-0.32,-0.30,-0.39,0.18,0.24,0.32
INDUS,0.40,-0.53,1.00,0.07,-0.39,-0.40,0.65,-0.71,0.58,0.71,0.37,-0.36,-0.46,-0.47
CHAS,-0.05,-0.04,0.07,1.00,-0.15,0.12,0.09,-0.11,0.01,-0.02,-0.12,0.05,0.08,0.19
NOX,-0.11,0.18,-0.39,-0.15,1.00,0.11,-0.31,0.30,-0.39,-0.40,-0.04,0.30,0.15,0.26
RM,-0.21,0.28,-0.40,0.12,0.11,1.00,-0.22,0.20,-0.21,-0.30,-0.34,0.12,0.34,0.68
AGE,0.34,-0.58,0.65,0.09,-0.31,-0.22,1.00,-0.76,0.44,0.50,0.26,-0.28,-0.45,-0.34
DIS,-0.37,0.67,-0.71,-0.11,0.30,0.20,-0.76,1.00,-0.49,-0.54,-0.23,0.30,0.38,0.23
RAD,0.63,-0.32,0.58,0.01,-0.39,-0.21,0.44,-0.49,1.00,0.91,0.47,-0.46,-0.44,-0.37
TAX,0.58,-0.30,0.71,-0.02,-0.40,-0.30,0.50,-0.54,0.91,1.00,0.46,-0.45,-0.46,-0.46


In [78]:
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.13960,0.0,8.56,0.0,1,6.167,90.0,2.4210,5,384.0,NaN,392.69,1,86.066034
1,0.03510,95.0,2.68,0.0,1,7.853,33.2,5.1180,4,224.0,NaN,392.78,1,207.700057
2,15.87440,0.0,18.10,0.0,1,6.545,99.1,1.5192,24,666.0,NaN,396.90,0,46.689349
3,0.18337,0.0,27.74,0.0,1,5.414,98.3,1.7554,4,711.0,NaN,344.05,0,29.980669
4,0.12816,12.5,6.07,0.0,1,5.885,33.0,6.4980,4,345.0,NaN,396.90,1,89.461275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0.17120,0.0,8.56,0.0,1,5.836,91.9,2.2110,5,384.0,20.9,395.67,0,83.647456
426,0.29916,20.0,6.96,0.0,1,5.856,42.1,4.4290,3,223.0,18.6,388.65,1,90.409128
427,0.01501,80.0,2.01,0.0,1,6.635,29.7,8.3440,4,280.0,17.0,390.94,1,104.921869
428,11.16040,0.0,18.10,0.0,0,6.629,94.6,2.1247,24,666.0,20.2,109.85,0,57.427501


In [79]:
X = df.drop(columns=['MEDV'])
y = df['MEDV']

In [80]:
X.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT'],
      dtype='object')

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [82]:
imputer = SimpleImputer(strategy="constant")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [116]:
from sklearn.svm import NuSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.ensemble import RandomForestRegressor

model = DecisionTreeRegressor(random_state=5)

final_model = model.fit(X_train, y_train)
y_pred1 = model.predict(X_test)

print(f"Model1 MSE: {mean_squared_error(y_test, y_pred1)}")

Model1 MSE: 372.7398928819352


In [117]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
print(f"Model RMSE: {np.sqrt(mean_squared_error(y_test, y_pred1))}")
print(f"Model MAE: {mean_absolute_error(y_test, y_pred1)}")

Model RMSE: 19.306472823432436
Model MAE: 13.47559233854493


In [118]:
df_valid = pd.read_csv("../data/03/housing/housing_validation.csv")
df_valid

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.09178,0.0,4.05,0.0,NaN,6.416,84.1,2.6463,5,296.0,NaN,395.50,NaN
1,0.05644,40.0,6.41,1.0,NaN,6.758,32.9,4.0776,4,254.0,NaN,396.90,NaN
2,0.10574,0.0,27.74,0.0,NaN,5.983,98.8,1.8681,4,711.0,NaN,390.11,18.07
3,0.09164,0.0,10.81,0.0,NaN,6.065,7.8,5.2873,4,305.0,NaN,390.91,NaN
4,5.09017,0.0,18.10,0.0,0.713,6.297,91.8,2.3682,24,666.0,NaN,385.09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,5.70818,0.0,18.10,0.0,NaN,6.750,74.9,3.3317,24,666.0,NaN,393.07,NaN
72,12.80230,0.0,18.10,0.0,0.740,5.854,96.6,1.8956,24,666.0,NaN,240.52,23.79
73,0.10659,80.0,1.91,0.0,NaN,5.936,19.5,10.5857,4,334.0,NaN,376.04,NaN
74,0.08707,0.0,12.83,0.0,NaN,6.140,45.8,4.0905,5,398.0,NaN,386.96,NaN


In [119]:
df_valid["NOX"] = df.NOX.isnull().astype("int")
df_valid["LSTAT"] = df.LSTAT.isnull().astype("int")

In [120]:
imputer = SimpleImputer(strategy = 'constant')
df_valid = imputer.fit_transform(df_valid)  
df_valid  

array([[9.17800e-02, 0.00000e+00, 4.05000e+00, 0.00000e+00, 0.00000e+00,
        6.41600e+00, 8.41000e+01, 2.64630e+00, 5.00000e+00, 2.96000e+02,
        0.00000e+00, 3.95500e+02, 0.00000e+00],
       [5.64400e-02, 4.00000e+01, 6.41000e+00, 1.00000e+00, 0.00000e+00,
        6.75800e+00, 3.29000e+01, 4.07760e+00, 4.00000e+00, 2.54000e+02,
        0.00000e+00, 3.96900e+02, 0.00000e+00],
       [1.05740e-01, 0.00000e+00, 2.77400e+01, 0.00000e+00, 0.00000e+00,
        5.98300e+00, 9.88000e+01, 1.86810e+00, 4.00000e+00, 7.11000e+02,
        0.00000e+00, 3.90110e+02, 0.00000e+00],
       [9.16400e-02, 0.00000e+00, 1.08100e+01, 0.00000e+00, 0.00000e+00,
        6.06500e+00, 7.80000e+00, 5.28730e+00, 4.00000e+00, 3.05000e+02,
        0.00000e+00, 3.90910e+02, 0.00000e+00],
       [5.09017e+00, 0.00000e+00, 1.81000e+01, 0.00000e+00, 0.00000e+00,
        6.29700e+00, 9.18000e+01, 2.36820e+00, 2.40000e+01, 6.66000e+02,
        0.00000e+00, 3.85090e+02, 0.00000e+00],
       [1.01530e-01, 0.00000e+

In [121]:
y_valid_predict = model.predict(df_valid)

In [122]:
y_valid_predict

array([ 76.28980246,  96.44659188,  47.17557907,  76.28980246,
        73.19050807,  76.28980246,  76.28980246,  76.28980246,
        73.19050807,  76.28980246,  87.81722255,  76.28980246,
        45.03868108,  76.28980246,  96.44659188, 107.24418131,
        87.81722255,  59.1302258 , 214.24663842,  73.19050807,
        96.44659188,  96.44659188,  61.32413737,  96.44659188,
        47.17557907,  55.75013176,  76.28980246,  60.38269582,
        71.18515015,  76.28980246,  76.28980246,  96.44659188,
        76.21052055,  73.19050807,  60.38269582,  59.15729521,
       141.52384185,  96.44659188,  96.44659188,  76.28980246,
        76.28980246, 150.23532226, 207.70005688,  76.28980246,
        96.44659188,  53.62232891,  61.32413737,  76.28980246,
        73.19050807,  76.28980246,  76.28980246, 149.45246865,
        76.28980246,  76.28980246, 207.70005688,  99.33223001,
        73.19050807, 149.45246865,  96.44659188,  76.28980246,
        96.44659188, 149.45246865,  76.28980246,  57.90

In [139]:
y_valid_predict = pd.DataFrame(y_valid_predict.round(2)).rename(columns={0: 'MEDV'})

In [141]:
y_valid_predict.to_csv("../data/results/housing_example_submission.csv", index=False)